In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import re
from nltk.util import ngrams

In [2]:
####for google colab
#!pip install -U sentence-transformers
#!pip install InstructorEmbedding


In [3]:
from sentence_transformers import SentenceTransformer
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('moka-ai/m3e-base')

load INSTRUCTOR_Transformer
max_seq_length  512


In [4]:
####if there is error, run the following and type Y as instructed and restart as instructed (for google colab)
#!pip uninstall sentence-transformers
#!pip install sentence-transformers==2.2.2

In [5]:
####################part 1. embedding

In [6]:
#####0. functions

In [7]:
def generate_ngrams(row, n):
    
    res= []
    
    if pd.isnull(row['评论内容']) == False:
    
        ngramss = ngrams( re.split('，|。|！', row['评论内容']), n)

        for grams in ngramss:
            if grams != '':
                tmp = ''
                for k in range(0, n):
                    tmp = tmp + grams[k] + '，'
                tmp = tmp[:-1]
                res.append(tmp)
        
    return res
            

In [8]:
def changeno(df):
    n=1
    for k in range(0,len(df)):
        if k == 0:
            prev =  df['评论数序号'].iloc[k]
            df.at[k,'no'] = n
        else:
            if prev == df['评论数序号'].iloc[k]:
                df.at[k,'no'] = n
            else:
                prev = df['评论数序号'].iloc[k] 
                n = n+1
                df.at[k,'no'] = n
    return (df)
        

In [9]:
######1. load the files and make embedding

In [10]:
####for query - n gram = 1
df = pd.read_excel('3.12阔腿裤客户评论.xlsx', sheet_name = 'ChatGPT抽取句子和关键词用（200）')
df['评论数序号'] = df['评论数序号'].ffill()
df = changeno(df)
df['评论内容_new'] = df.apply(generate_ngrams, axis=1,n=1)
df = df.explode('评论内容_new')
query_df = df[['评论数序号','评论内容','评论内容_new','no']].dropna()
query_df['query_embedding'] = query_df['评论内容_new'].apply(lambda x: model.encode([x]))
query_df.to_pickle('query_df.pkl')
query_df

,评论数序号,评论内容,评论内容_new,no,query_embedding
1,1.0,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,非常舒适,1.0,"[[0.08720997, 0.700452, 0.38203034, -0.4097930..."
1,1.0,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,版型大气,1.0,"[[0.011770149, 0.73211724, 0.088851355, 0.2304..."
1,1.0,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,颜色正,1.0,"[[0.20295629, 1.3116555, 0.5653437, -0.8313223..."
1,1.0,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,冬天穿,1.0,"[[0.45391983, 1.6987015, 0.34006888, -0.163527..."
1,1.0,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,御寒功能强,1.0,"[[0.41413492, 1.2071105, 0.29200175, 0.5620494..."
...,...,...,...,...,...
626,395.0,【尺码大小】裤长有丢丢长,【尺码大小】裤长有丢丢长,199.0,"[[0.2243332, 1.7851044, 0.46404594, 0.07525892..."
627,395.0,柔软厚实 裤型很挺括,柔软厚实 裤型很挺括,199.0,"[[0.66592747, 1.392358, 0.8201952, 0.6959012, ..."
629,396.0,【尺码大小】颜色奶白色，好看帕罗品牌高大上，每件都喜欢,【尺码大小】颜色奶白色,200.0,"[[0.22040766, 1.4115331, 0.35335177, -0.010529..."
629,396.0,【尺码大小】颜色奶白色，好看帕罗品牌高大上，每件都喜欢,好看帕罗品牌高大上,200.0,"[[1.3612443, 0.7273756, 0.13875581, -0.5038888..."


In [11]:
####for query - n gram = 2
df = pd.read_excel('3.12阔腿裤客户评论.xlsx', sheet_name = 'ChatGPT抽取句子和关键词用（200）')
df['评论数序号'] = df['评论数序号'].ffill()
df = changeno(df)
df['评论内容_new2'] = df.apply(generate_ngrams, axis=1,n=2)
df = df.explode('评论内容_new2')
query_df = df[['评论数序号','评论内容','评论内容_new2','no']].dropna()
query_df['query_embedding'] = query_df['评论内容_new2'].apply(lambda x: model.encode([x]))
query_df.to_pickle('query_df2.pkl')
query_df

,评论数序号,评论内容,评论内容_new2,no,query_embedding
1,1.0,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,非常舒适，版型大气,1.0,"[[0.39708078, 0.5066099, -0.030516662, -0.2275..."
1,1.0,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,版型大气，颜色正,1.0,"[[0.14094841, 0.88311005, 0.1986682, -0.296296..."
1,1.0,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,颜色正，冬天穿,1.0,"[[0.1534357, 1.726912, 0.44430065, -0.6705892,..."
1,1.0,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,冬天穿，御寒功能强,1.0,"[[0.3712388, 1.3276333, 0.15939693, 0.06523707..."
1,1.0,非常舒适，版型大气，颜色正！冬天穿，御寒功能强，配大衣和羽绒服都合适。,御寒功能强，配大衣和羽绒服都合适,1.0,"[[0.5275978, 1.2394882, 0.02691713, -0.0490422..."
...,...,...,...,...,...
621,393.0,【尺码大小】尺码标准，140穿这个码正合适,【尺码大小】尺码标准，140穿这个码正合适,198.0,"[[0.6146101, 1.5653566, 0.16415079, -0.0623720..."
622,393.0,面料柔软，裤型好看。,面料柔软，裤型好看,198.0,"[[0.50281197, 1.1825496, 0.26331162, 0.1434267..."
622,393.0,面料柔软，裤型好看。,裤型好看，,198.0,"[[-0.084541984, 1.472511, 0.24696197, -0.13317..."
629,396.0,【尺码大小】颜色奶白色，好看帕罗品牌高大上，每件都喜欢,【尺码大小】颜色奶白色，好看帕罗品牌高大上,200.0,"[[0.98055863, 0.9319363, 0.19641289, -0.352272..."


In [12]:
#### faq
df = pd.read_excel('3.8最新new_faq.xlsx')
df['faq_new'] = df['分类结果'] + ':' + df['分类关键句_new']
df['faq_embedding'] = df['faq_new'].apply(lambda x: model.encode([x]))
df.to_pickle('faq_df.pkl')
df

,分类结果,分类关键句_new,faq_new,faq_embedding
0,产品-保暖性-好,厚裤子，非常暖和,产品-保暖性-好:厚裤子，非常暖和,"[[0.7471114, 1.6507893, 0.4053107, 0.1798715, ..."
1,产品-保暖性-好,面料厚实合适,产品-保暖性-好:面料厚实合适,"[[1.0033041, 1.5493777, 0.657772, -0.047733348..."
2,产品-保暖性-好,加绒厚实，比较保暖,产品-保暖性-好:加绒厚实，比较保暖,"[[0.74150646, 1.6728475, 0.4939154, 0.18470088..."
3,产品-保暖性-好,很暖和，内里有绒,产品-保暖性-好:很暖和，内里有绒,"[[0.78655845, 1.5695466, 0.8508352, -0.0880344..."
4,产品-保暖性-好,厚度适中,产品-保暖性-好:厚度适中,"[[0.549692, 1.3765683, 0.8885594, -0.14377633,..."
...,...,...,...,...
200,服务-物流-慢,对于物流速度有点失望,服务-物流-慢:对于物流速度有点失望,"[[0.6244018, 0.8205226, 0.5795652, -0.09391755..."
201,服务-物流-慢,裤子的到货时间比我预想的晚了不少,服务-物流-慢:裤子的到货时间比我预想的晚了不少,"[[0.734299, 0.8220778, 0.25637123, -0.20384425..."
202,服务-物流-慢,但物流方面确实有点慢，等待的过程有些煎熬,服务-物流-慢:但物流方面确实有点慢，等待的过程有些煎熬,"[[0.69423395, 0.35844174, 0.4791915, -0.112863..."
203,服务-物流-慢,这次的物流时间比我以往的购物经历要长,服务-物流-慢:这次的物流时间比我以往的购物经历要长,"[[0.68231213, 0.5811415, 0.6342791, -0.0915558..."
